# **Compressing circuit in .qasm files**

In [30]:
import sys
sys.path.insert(0, "../..") # clue is here
from clue import *
from clue.qiskit import *
from numpy import load, save
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer, qpy
from qiskit.quantum_info import Operator
from time import time

In this document we present the code necessary to transform a ".qasm" file from its original shape to a compressed version that can do the same computations taking into consideration the corresponding lumping. 

The main idea behind this notebook is to take a `.qasm` file, create the corresponding quantum circuit using the unitary matrix associated to it and then compute a lumping for the input $\ket{0}$ which is the usual input for these circuits. Then, after obtaining the new unitary matrix, we will build a new circuit (hopefully smaller) using this matrix and save it as a new `.qasm` file.

Finally, we would like to compare the simulation time. The measurement is slightly tricky in the reduced circuit since we need to know the value of the associated vector that is represented by each base state in the reduced circuit and perform one last measurment there.

## 1. Lumping and storing the reduction

Here we provide a code that allows to reduce a quantum circuit into a (hopefully) smaller representation of it. It stores on memory the reduced version using a unitary matrix and the lumping matrix to recover later the final measurement.

In [31]:
def reduce_circuit(folder: str, filename: str):
    ctime = time()
    ## Reading the ".qasm" circuit
    circuit = DS_QuantumCircuit.from_qasm_file(f"{folder}/{filename}")
    read_time=time()-ctime; ctime = time()
    ## Computing the lumping for the first state
    lumped = circuit.lumping([circuit.variables[0]], print_reduction=False, print_system=False)
    lumping_time=time()-ctime; ctime=time()
    ## Writing the reduced circuit
    Ur = lumped.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble); Ur
    Ur = extend_to_power(Ur)
    nqbits = int(log2(Ur.shape[0]))
    q = QuantumRegister(nqbits,'q')
    c = ClassicalRegister(nqbits,'c')
    red_circuit = QuantumCircuit(q,c)
    red_circuit.unitary(Ur, q)
    red_circuit.measure(q, c)
    with open(f"{folder}/reduced/{filename}.qpy", "wb") as f:
        qpy.dump(red_circuit, f)
    reduced_time = time()-ctime; ctime=time()
    ## Writing the lumping matrix
    save(f"{folder}/reduced/{filename}.npy", lumped.lumping_matrix.to_numpy(dtype=cdouble))
    matrix_time = time()-ctime
    
    return {"read": read_time, "lumping": lumping_time, "reduced": reduced_time, "matrix": matrix_time}
    

In [32]:
reduce_circuit("./circuits", "grover-noancilla_indep_qiskit_7.qasm")

{'read': 0.4943397045135498,
 'lumping': 0.6573669910430908,
 'reduced': 0.0007796287536621094,
 'matrix': 0.0005483627319335938}

## 2. Simulating a quantum circuit

Here we provide a piece of code that receives how many times to simulate a system ans uses the qiskit framework to do it. It will return the distribution of the outputs

In [33]:
def simulate_qasm(folder: str, filename:str, shots=8192):
    ctime = time()
    ## Reading the quantum circuit
    circuit = QuantumCircuit.from_qasm_file(f"{folder}/{filename}")
    reading_time = time()-ctime; ctime=time()
    ## Simulating the quantum circuit
    backend = Aer.get_backend('aer_simulator')
    job = execute(circuit, backend, shots=shots)
    simulation_time = time()-ctime; ctime=time()
    ## Collecting the data to have proper output
    output = job.result().get_counts()
    output_time = time()-ctime
    
    return output, {"read": reading_time, "simulation": simulation_time, "output": output_time}

In [37]:
res = simulate_qasm("./circuits", "grover-noancilla_indep_qiskit_7.qasm")
print(res[1])
print(sum(res[1].values()))

{'read': 0.11484146118164062, 'simulation': 0.02226734161376953, 'output': 0.013597726821899414}
0.15070652961730957


In [38]:
res[0]

{'1111101': 1,
 '1001101': 1,
 '1111010': 1,
 '1001001': 1,
 '1100010': 1,
 '1111000': 1,
 '1111100': 1,
 '1100100': 1,
 '1010100': 2,
 '1111111': 8154,
 '1001111': 1,
 '1110111': 1,
 '1101010': 3,
 '1000000': 3,
 '1101001': 3,
 '1101110': 2,
 '1100001': 2,
 '1101100': 2,
 '1101101': 1,
 '1100000': 1,
 '1011100': 3,
 '1010111': 1,
 '1011011': 1,
 '1011000': 1,
 '1001010': 1,
 '1001110': 1,
 '1001011': 1}

## 3. Simulating a reduced quantum circuit

In this case, we assume the `.qasm` file has associated a `.npy` file with the corresponding lumping needed for transforming the measure of the `.qasm` circuit to the actual output of the circuit. This adds a small overload to the simulation. The output will be similar to that in `simulate_qasm`.

In [35]:
def simulate_reduced(folder: str, filename:str, shots=8192):
    ctime = time()
    ## Reading the quantum circuit
    with open(f"{folder}/{filename}", "rb") as f:
        circuit = qpy.load(f)[0]
    reading_time = time()-ctime; ctime=time()    
    ## Simulating the quantum circuit
    backend = Aer.get_backend('aer_simulator')
    job = execute(circuit, backend, shots=shots)
    simulation_time = time()-ctime; ctime=time()
    ## Collecting the data to have proper output
    #### Reading the lumping matrix
    Ur = load(f"{folder}/{filename.removesuffix('.qpy')}.npy")
    nqbits = int(log2(Ur.shape[1]))
    matrix_time = time()-ctime; ctime=time()
    #### Measuring the data from qasm to the original states
    output = dict()
    for (out, times) in job.result().get_counts().items():
        for m,t in repeated_measure(Ur[int(out, 2)], times, out=dict).items():
            output[m] = output.get(m,0) + t
    output = {format(k, f"0{nqbits}b"): v for k,v in output.items()}
    measuring_time = time()-ctime
    output_time = matrix_time + measuring_time
    
    return output, {"read": reading_time, "simulation": simulation_time, "output": {"total": output_time, "matrix": matrix_time, "measure": measuring_time}}

In [39]:
res = simulate_reduced("./circuits/reduced", "grover-noancilla_indep_qiskit_7.qasm.qpy")
print(res[1])
print(sum(el if not isinstance(el, dict) else el["total"] for el in res[1].values()))

{'read': 0.0010499954223632812, 'simulation': 0.0019762516021728516, 'output': {'total': 0.009523630142211914, 'matrix': 0.007335186004638672, 'measure': 0.002188444137573242}}
0.012549877166748047


In [40]:
res[0]

{'1110000': 119,
 '1111010': 120,
 '1011100': 131,
 '1111110': 128,
 '1001111': 115,
 '1011010': 126,
 '1101100': 134,
 '1010010': 154,
 '1111000': 148,
 '1001010': 113,
 '1000011': 110,
 '1010101': 119,
 '1011001': 112,
 '1111111': 143,
 '1001101': 138,
 '1101101': 134,
 '1011011': 135,
 '1010110': 112,
 '1110001': 126,
 '1101001': 119,
 '1000000': 129,
 '1111100': 112,
 '1001000': 133,
 '1000111': 138,
 '1000101': 124,
 '1100101': 116,
 '1010100': 135,
 '1110101': 114,
 '1100011': 124,
 '1011101': 122,
 '1010111': 107,
 '1001011': 130,
 '1101011': 122,
 '1000001': 133,
 '1101110': 143,
 '1010011': 128,
 '1100111': 137,
 '1100001': 162,
 '1110110': 134,
 '1101010': 139,
 '1000110': 122,
 '1110010': 134,
 '1111011': 133,
 '1011110': 132,
 '1010001': 104,
 '1001100': 117,
 '1000100': 119,
 '1010000': 135,
 '1101111': 121,
 '1001001': 130,
 '1101000': 152,
 '1110111': 133,
 '1110100': 131,
 '1100010': 137,
 '1110011': 129,
 '1100100': 142,
 '1111001': 138,
 '1011111': 122,
 '1000010': 13

In [ ]:
circuit